In [1]:
import numpy as np

In [ ]:
class Node:
         
    def __init__(self, initial, parent=None, acao=None, custo=0):

        self.initial = initial
    
    def __repr__(self):
        return "<Node {}>".format(self.initial)

    def expansao(self, problema, rainha=None):

        if rainha == None:
            rainha_list = self.encontrar_rainha()
            sorteio = np.random.choice(range(len(rainha_list)))
            rainha = rainha_list[sorteio]
        
        print(rainha)
        
        return [self.child_node(problema, rainha, acao) 
                for acao in problema.pos_mov()]
        

    def child_node(self, problem, rainha, acao):

        prox_estado = problem.jogada(rainha, acao)

        prox_node = Node(prox_estado, self, acao, 0)

        return prox_node
    
    def encontrar_rainha(self):

        coordenadas = []
        fil = len(self.initial)
        col = len(self.initial[:1][0])
        for i in range(fil):
            for j in range(col):
                if self.initial[i][j] == 1:
                    coordenadas.append([i,j])
                    print('i')
        return coordenadas

In [ ]:
class oitorainhas:

    def __init__(self, x, y, tabuleiro=None):
        self.x = x
        self.y = y

        if tabuleiro == None:
            self.tabuleiro = np.zeros([x,y])

            for i in range(x):
                self.tabuleiro[i][0] = 1

    def pos_mov(self, tabuleiro):

        pos = []
        for i in range(self.x):
            for j in range(self.y):
                if tabuleiro[i][j] != 1:
                    pos.append([i,j])
        
        return pos

    def jogada(self, tabuleiro, rainha, coordenada):

        tab = tabuleiro.copy()

        tab[rainha[0],rainha[1]] = 0
        tab[coordenada[0],coordenada[1]] = 1

        return tab
    
    def test_objetivo(self, tabuleiro):

        invalido = 0

        for icol in range(self.y):

            for irow in range(self.x):
                
                # -1 pois iremos ignorar a contagem do encontrado
                # Contagem de 1s na fileira
                fil = -1
                # Contagem de 1s na coluna
                col = -1
                # Contagem de 1s na diagonal direita
                dig_dir = -1
                # Contagem de 1s na diagonal esquerda
                dig_esq = -1

                if(tabuleiro[irow][icol] == 1):

                    # Contar 1s na coluna icol
                    for k in range(self.y):
                        if (tabuleiro[irow][k] == 1):
                            fil+=1

                            invalido+=1

                    # Contar 1s na fileira irow
                    for k in range(self.x):
                        if (tabuleiro[k][icol] == 1):
                            col+=1

                            invalido+=1

                    # encontrar diagonal esquerda (secundaria)
                    col_esq = icol
                    row_esq = irow
                    while(col_esq != 0 and row_esq != (self.x-1)):
                        col_esq-=1
                        row_esq+=1       

                    # cencontrar diagonal direita (primaria)
                    col_dir = icol
                    row_dir = irow
                    while(col_dir != (self.y-1) and row_dir != (self.x-1)):
                        col_dir+=1
                        row_dir+=1

                    # contar 1's diagonal esquerda (secundaria)
                    while(row_esq >= 0 and col_esq <= self.y-1):

                        if (tabuleiro[row_esq][col_esq] == 1):
                            dig_esq+=1

                            invalido+=1

                        col_esq+=1
                        row_esq-=1

                    # contar 1's diagonal direita (primaria)
                    while(col_dir>=0 and row_dir>=0):

                        if (tabuleiro[row_dir][col_dir] == 1):
                            dig_dir+=1

                            invalido+=1
                            
                        col_dir-=1
                        row_dir-=1

                    # Deverão ser contados as marcações originais
                    if invalido <= self.x:
                        return True
                    else:
                        return False